Chargement des données

In [1]:
import numpy as np
import cv2
import glob
import os
import matplotlib.pyplot as plt
from tqdm import tqdm

def load_images(path):
    image_files = glob.glob(path)
    images = []
    for file in image_files:
        img = cv2.imread(file)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        # Prétraitement si nécessaire
        images.append(img)
    return np.array(images)

# Spécifier le chemin vers le répertoire contenant les images
input_train_path = os.getcwd()+"/Paires d'images"
print(f"input path : {input_train_path}")

class_names = ['11.369', '16.109', '43.446']

# Charger les images pour l'entraînement
train_val_image_file_list = []
train_val_label_list = []

for i in range(len(class_names)):
    for filename in glob.iglob( os.path.join(input_train_path,class_names[i]) + '/**/*.jpg', recursive=True):
        # extract patient number and slice
        train_val_image_file_list.append(filename)
        train_val_label_list.append( i )     

print(len(train_val_image_file_list), len(train_val_label_list))
print(train_val_image_file_list[0])


input path : /home/abarras/Documents/TDSI/Projet Alstom/Paires d'images
6 6
/home/abarras/Documents/TDSI/Projet Alstom/Paires d'images/11.369/20230323_191223_BRS_L_UP_11.369_00.jpg


Création du modèle de réseau de neurones

In [2]:

from PIL import Image, ImageFilter
# Reading images 
def ReadImages(images_files):
    X = []
    for index in tqdm(range(len(images_files))):
        #image_read = cv2.imread(images_files[index], cv2.IMREAD_COLOR)
        image_read = Image.open(images_files[index])
       
    X = np.asarray(X, dtype=np.uint8)
    return X

# Read images
X_train_rgb = ReadImages(train_val_image_file_list)

100%|██████████| 6/6 [00:00<00:00, 472.74it/s]


In [3]:
def adjust_brightness(image, factor):
    hsv = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
    hsv[:, :, 2] = np.clip(hsv[:, :, 2] * factor, 0, 255)
    return cv2.cvtColor(hsv, cv2.COLOR_HSV2RGB)

def build_brightness_model(input_shape):
    model = models.Sequential()
    model.add(layers.Flatten(input_shape=input_shape))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(1, activation='linear'))  # Linear activation for regression
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model


In [4]:
def read_images_with_brightness(images_files, brightness_model):
    X = []
    y_brightness = []

    # Load and preprocess the first image separately
    first_image_read = cv2.imread(images_files[0])
    first_image_rgb = cv2.cvtColor(first_image_read, cv2.COLOR_BGR2RGB)
    first_image_brightness = 1.0  # Initial brightness factor for the first image

    # Add the first image to the data
    X.append(first_image_rgb)
    y_brightness.append(first_image_brightness)

    # Train the brightness adjustment model using the first image
    for index in tqdm(range(1, len(images_files))):
        image_read = cv2.imread(images_files[index])
        image_rgb = cv2.cvtColor(image_read, cv2.COLOR_BGR2RGB)

        # Additional preprocessing if needed, e.g., resizing, normalization
        # ...

        # Extract features from the first image (brightness in this case)
        first_image_features = [first_image_brightness]

        # Predict brightness using the trained model
        brightness_factor = brightness_model.predict(np.asarray([first_image_features]))[0, 0]

        # Adjust brightness
        adjusted_image = adjust_brightness(image_rgb, brightness_factor)

        X.append(adjusted_image)
        y_brightness.append(brightness_factor)

    X = np.asarray(X, dtype=np.uint8)
    y_brightness = np.asarray(y_brightness, dtype=np.float32)

    return X, y_brightness

In [6]:
# Load and preprocess images
X_train_rgb = load_images(os.path.join(input_train_path, class_names[0], '*.jpg'))

# Load brightness labels (you need to create y_train_brightness based on your data)
y_train_brightness = ...

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train_rgb, y_train_brightness, test_size=0.2, random_state=42)

# Build the brightness adjustment model
input_shape = (1,)  # Features from the first image (brightness in this case)
brightness_model = build_brightness_model(input_shape)

# Train the brightness adjustment model
brightness_model.fit(np.asarray([[1.0]]), np.asarray([1.0]), epochs=10, batch_size=1, verbose=0)

# Read images and adjust brightness using the trained model
X_train_adjusted, _ = read_images_with_brightness(train_val_image_file_list, brightness_model)


/tmp/ipykernel_20630/1837279618.py:16: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(images)


NameError: name 'train_test_split' is not defined

In [ ]:
image_read = Image.open(train_val_image_file_list[0])

In [ ]:
from tensorflow.keras.layers import Conv2D, Input, UpSampling2D, Lambda
from tensorflow.keras.models import Model

def create_brightness_adjustment_model(input_shape):
    input_img = Input(shape=input_shape)
    # Définition du modèle...
    
    # Exemple de modèle simple pour l'ajustement de luminosité (à adapter selon les besoins)
    conv_1 = Conv2D(64, (3, 3), activation='relu', padding='same')(input_img)
    upsample_1 = UpSampling2D((2, 2))(conv_1)
    decoded_img = Conv2D(3, (3, 3), activation='sigmoid', padding='same')(upsample_1)

    model = Model(inputs=input_img, outputs=decoded_img)
    return model

# Créer le modèle
input_shape = train_val_image_file_list[0].shape  # Utiliser la forme de la première image pour définir l'entrée du modèle
brightness_model = create_brightness_adjustment_model(input_shape)

# Compiler le modèle
brightness_model.compile(optimizer='adam', loss='mean_squared_error')

Entrainement du modèle

In [ ]:
# Entraînement du modèle
brightness_model.fit(X_train, X_train, epochs=10, batch_size=32, validation_split=0.2)

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models
from PIL import Image

# Charger les données et les étiquettes de luminosité (les étiquettes dépendront de votre ensemble de données)
# X_train, y_train = chargement_des_images_et_des_etiquettes()
path_data = os.getcwd()+'/BDD étiquettes.csv'
data = pd.read_csv(path_data)

X_train_paths = data['Image']
y_train = data['Etiquette']

# Charger le chemin de la première image dans X_train_paths
chemin_image = X_train_paths[0]

# Charger l'image à partir du chemin spécifié
image = Image.open(chemin_image)

# Prétraitement des données
# Normaliser les valeurs des pixels, redimensionner les images, etc.
# X_train = pretraitement_des_images(X_train)

# Construire le modèle CNN
# model = models.Sequential()
# model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(largeur_image, hauteur_image, canaux)))
# model.add(layers.MaxPooling2D((2, 2)))
# model.add(layers.Conv2D(64, (3, 3), activation='relu'))
# model.add(layers.MaxPooling2D((2, 2)))
# model.add(layers.Conv2D(128, (3, 3), activation='relu'))
# model.add(layers.MaxPooling2D((2, 2)))
# model.add(layers.Flatten())
# model.add(layers.Dense(128, activation='relu'))
# model.add(layers.Dense(1, activation='linear'))  # Couche de sortie pour la prédiction de luminosité

# Compiler le modèle
# model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

# Entraîner le modèle
# model.fit(X_train, y_train, epochs=nb_epochs, batch_size=batch_size, validation_split=0.2)

# Afficher la première image de la base de données
plt.imshow(X_train_paths[0])
plt.axis('off')  # Masquer les axes
plt.title('Première image de la base de données')
plt.show()